In [ ]:
print("hi")

import os
import pandas as pd
import subprocess
import time
import re
import numpy as np

import csv

from scripts import combat_info
from scripts import combat_quick_apply
from scripts import combat_quick_QC


models_directory = "ADNI_MODELS"
raw_directory = 'RAW/ADNI'
clean_directory = 'ADNI_CLEAN'
best_directory = 'ADNI_BEST'

method= "classic"

In [2]:
def get_info(mov_data_file):
    [df,bundles] = combat_info.info(mov_data_file)
    nb_hc = int(re.findall('HC\(n=(\d+)',df["DetailInfos"]["Disease"])[0])
    nb_total = df["DetailInfos"]["Number of Subject"]
    nb_sick = nb_total - nb_hc
    return [nb_total,nb_hc,nb_sick]

In [3]:
def get_bundles(mov_data_file):
    return combat_info.get_bundles(mov_data_file)

In [4]:
def fit(mov_data_file, CAMCAN, metric):
    mov_data = pd.read_csv(mov_data_file)
    ###########
    ### fit ###
    ###########
    output_model_filename = (
            mov_data.site.unique()[0]
            + "."
            + metric
            + "."
            + method
            + ".model.csv"
        )
    cmd = (
        "scripts/combat_quick_fit.py"
        + " "
        + CAMCAN
        + " "
        + mov_data_file
        + " --out_dir "
        + models_directory
        + " --output_model_filename "
        + output_model_filename
        + " --method "
        + method
        + " --hc "
    )
    subprocess.call(cmd, shell=True)
    return

In [5]:
def apply(mov_data_file, metric):
    mov_data = pd.read_csv(mov_data_file)
    clean_mov_data = (
        clean_directory
        +
        '/'
        +
        mov_data.site.unique()[0]
        + "."
        + metric
        + "."
        + method
        + ".csv.gz"
    )
    model = (
            models_directory
            +
            '/'
            +
            mov_data.site.unique()[0]
            + "."
            + metric
            + "."
            + method
            + ".model.csv"
        )
    combat_quick_apply.apply(mov_data_file, model, clean_mov_data)

In [6]:
def QC(mov_data_file, CAMCAN, metric):
    mov_data = pd.read_csv(mov_data_file)

    model_file = (
            models_directory
            +
            '/'
            +
            mov_data.site.unique()[0]
            + "."
            + metric
            + "."
            + method
            + ".model.csv"
    )
    clean_mov_data = (
        clean_directory
        +
        '/'
        +
        mov_data.site.unique()[0]
        + "."
        + metric
        + "."
        + method
        + ".csv.gz"
    )
    return combat_quick_QC.QC(CAMCAN,clean_mov_data, model_file)

In [7]:
def get_csv_res(mov_data_file, metric):
    mov_data = pd.read_csv(mov_data_file)
    model = (
            models_directory
            +
            '/'
            +
            mov_data.site.unique()[0]
            + "."
            + metric
            + "."
            + method
            + ".model.csv"
        )
    return combat_quick_apply.make_best(mov_data_file, model)

In [8]:
# mat =[]
# fields = ["Name","NB Suject","NB HC","NB Sick","Mean Bhattacharrya"]

# metric = "md"
# CAMCAN = f"./DONNES/CamCAN.{metric}.raw.csv.gz"

# for filename in sorted(os.listdir(raw_directory)):
#     f = os.path.join(raw_directory, filename)
#     # checking if it is a file
#     if os.path.isfile(f):
#         if len(fields)==5:
#             fields = fields + get_bundles(f)

#         print(f)
#         infos = [filename]  
#         infos = infos + get_info(f)
#         fit(f, CAMCAN, metric)
#         apply(f, metric)
#         dist = QC(f, CAMCAN, metric)
#         infos.append(np.mean(dist))
#         infos = infos + dist
#         mat.append(infos)
#         #print("---------------------------")
# print(mat)


In [9]:
# with open(f'GFG.{metric}', 'w') as res:
     
#     # using csv.writer method from CSV package
#     write = csv.writer(res)
     
#     write.writerow(fields)
#     write.writerows(mat)

In [10]:
sites = [
    "adni_2_Siemens_3T",
    "adni_27_GE_3T",
    "adni_29_GE_3T",
    "adni_41_Siemens_3T",
    "adni_94_GE_3T",
    "adni_98_GE_3T",
    "adni_99_GE_3T",
    "adni_127_GE_3T",
    "adni_129_GE_3T",
    "adni_141_Siemens_3T",
    "adni_941_Siemens_3T"
]
metrics = [
    "ad",
    "adt",
    "afd",
    "fa",
    "fat",
    "fw",
    "md",
    "mdt",
    "rd",
    "rdt"
]

In [ ]:
# header = (
#         "sid,bundle,metric,mean,site,age,sex,handedness,disease"
#     )

# for metric in metrics:
#     combination = np.empty((9, 0))
#     CAMCAN = f"./RAW/CamCAN/CamCAN.{metric}.raw.csv.gz"
#     for site in sites:
#         f = os.path.join(raw_directory, metric,  site + "." + metric + ".raw.csv.gz")
#         # checking if it is a file
#         if os.path.isfile(f):
#             print(f)
#             fit(f, CAMCAN, metric)
#             site = get_csv_res(f, metric)
#             combination = np.hstack([combination, site])
#             print("---------------------------")

#     np.savetxt(f"DONNES/adni_compilation.{metric}.csv.gz", combination.T, delimiter=",", header=header, fmt="%s", comments="")


In [11]:
# for filename in sorted(os.listdir(best_directory)):
#     f = os.path.join(best_directory, filename)
#     # checking if it is a file
#     if os.path.isfile(f):
#         print(f)
#         dir = "ADNI_BEST_VIZUALIZE/" + filename.split('.')[0]
#         cmd = (
#             "combat_visualize_data.py"
#             + " "
#             + f
#             + " "
#             + CAMCAN
#             + " --out_dir "
#             + dir
#             + " --bundles all"
#         )
#         subprocess.call(cmd, shell=True)
#         print("---------------------------")

In [ ]:
# for metric in metrics:
#     # Define the file
#     filename = f'DONNES/adni_compilation.{metric}.csv.gz'
#     CAMCAN = f"./RAW/CamCAN/CamCAN.{metric}.raw.csv.gz"
#     # Check if it is a file
#     if os.path.isfile(filename):
#         print(filename)
#         dir = f"ADNI_COMPILATION_VIZUALIZE/{metric}"
#         cmd = (
#             "combat_visualize_data.py"
#             + " "
#             + filename
#             + " "
#             + CAMCAN
#             + " --out_dir "
#             + dir
#             + " --bundles all"
#         )
#         subprocess.call(cmd, shell=True)
#         print("---------------------------")